# <font color='darkblue'>DeepHorse Model</font>

<img src="sha-tin-racecourse-slider-pic-mobile-02.jpg">


## Notebook summary:  
* Data pre-processing including labelling logic
* Helper functions
* DeepHorse v1
    * FC model with custom depth/units
    * Metrics
    * Hyperparam search
* Benchmarks
    * Uniform model
    * Public odds model
* DeepHorse v2
    * Proof-of-concept end-to-end version
    * Scratch-pad exploring Conv2D version

In [ ]:
import os
os.chdir('C:\\Users\\Olaf\\OneDrive\\Work\\StanfordAI\\Project\\Data\\spp')

import pandas as pd
import numpy as np
np.set_printoptions(precision=3, suppress=True)
np.random.seed(888)

import tensorflow as tf
tf.random.set_seed(888)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Dropout, Conv2D 
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
from keras.initializers import glorot_uniform

import datetime
import pydot

tf.__version__

## Data pre-processing

Note R and X_raw require a different set of pre-processing

### R: Feature-enginered data. Load data, extract features and place-vectors
Actual feature engineering on raw data is done in `DeepHorse_feature_engineering.ipynb`

In [ ]:
R = pd.read_csv('R.csv', index_col=0)
R_original = R.copy()
R_original.columns

In [ ]:
R_features = R_original.copy()
R_features.drop(labels=['race_id', 'horse_num'], axis=1, inplace=True) # useless
# R_features.drop(labels=['win_odds'], axis=1, inplace=True) # strong indicator, experiment with dropping

n_h = 14
n_rows = int(R_features.shape[0]/n_h)

R_place = R_features.pop('place')
R_place = np.array(R_place).astype(float)
R_place = R_place.reshape((int(n_rows),n_h))

numeric_cols = R_features.select_dtypes(include=np.number).columns.tolist()

n_f = len(numeric_cols)
R_features = R_features[numeric_cols]
R_features = np.array(R_features)
R_features = R_features.reshape((int(n_rows), n_h, n_f))

print('Features shape =', R_features.shape)
print('Labels shape =', R_place.shape)
print('Num features =', len(numeric_cols))
print('Feature names =', numeric_cols)

### X_raw: Load data and reformat

In [ ]:
Xraw = pd.read_csv('X_raw.csv', index_col=0)
Xraw_original = Xraw.copy()
Xraw_original.columns

In [ ]:
Xraw_features = Xraw_original.copy()
Xraw_features.drop(labels=['race_id', 'horse_num'], axis=1, inplace=True) # useless
# R_features.drop(labels=['win_odds'], axis=1, inplace=True) # strong indicator, experiment with dropping

n_h = 14
n_X_rows = int(Xraw_features.shape[0]/n_h)

Xraw_place = Xraw_features.pop('place')
Xraw_place = np.array(Xraw_place).astype(float)
Xraw_place = Xraw_place.reshape((int(n_X_rows),n_h))

numeric_cols = Xraw_features.select_dtypes(include=np.number).columns.tolist()

n_X_f = len(numeric_cols)
Xraw_features = Xraw_features[numeric_cols]
Xraw_features = np.array(Xraw_features)
Xraw_features = Xraw_features.reshape((int(n_X_rows), n_h, n_X_f))

print('Features shape =', Xraw_features.shape)
print('Labels shape =', Xraw_place.shape)
print('Num features =', len(numeric_cols))
# print('Feature names =', numeric_cols)

### Set labels from "place"
Vector of horse places, such that horse_i is in 1-14, replaced by all zeros except a weight on first/second/third place

In [ ]:
def set_labels(w1, w2, w3, labels):
    y = labels.copy() 
    y[y == 1] = w1
    y[y == 2] = w2
    y[y == 3] = w3
    y[y >= 4] = 0
    return y

w1 = .5
w2 = 0.25
w3 = 0.25

R_labels = set_labels(w1, w2, w3, R_place)

In [ ]:
Xraw_labels = set_labels(w1, w2, w3, Xraw_place)

### Train/Traindev/Dev/Test split

**General principle:** 

We are building a model to forecast races that will happen in the future. The distribution is expected to change through time for several reasons:
* The population of horses / jockeys / trainers changes over time
* The ranking/handicapping methodology of the HKJC may be subject to change
* Systematic climatic variation (e.g. unusually hot summer)
* Etc

Therefore the model is trained on race data **before** some cutoff date T and dev/test on race data **after** T.

**Method:**
* First split chronologically:
  * Dev&Test is the most recent races available, e.g. to use the *2019-2020* season, we set m_dev_test = 800
  * Train&Traindev is what comes before chronologically

* Next sample randomly:
    * **Dev** and **Test** sets are sampled randomly from the Dev&Test section

    * **Train** and **Train/Dev** are sampled randomly from the Train&Traindev section



In [ ]:
m_dev_test = 2000 #how many examples in the dev+test sections; 1600=2 most recent seasons
train_split = 1. #how to split the train&traindev section into train and traindev
dev_split = 0.5 #how to split the dev&test section into dev and test

### R

In [ ]:
X = R_features.copy()
y = R_labels.copy()
np.random.seed(888)

m_train_traindev = n_rows - m_dev_test

# Dev/Test sets are at the end chronologically
X_train_traindev = X[ : m_train_traindev]
y_train_traindev = y[ : m_train_traindev]
X_dev_test  = X[m_train_traindev : n_rows]
y_dev_test  = y[m_train_traindev : n_rows]

# Now can shuffle 
idx1 = np.random.permutation(m_train_traindev)
X_train_traindev, y_train_traindev = X_train_traindev[idx1], y_train_traindev[idx1]

idx2 = np.random.permutation(m_dev_test)
X_dev_test, y_dev_test = X_dev_test[idx2], y_dev_test[idx2]

# Split Train and Train/Dev sets
m_train = int(m_train_traindev * train_split)
X_train = X_train_traindev[ : m_train]
y_train = y_train_traindev[ : m_train]
X_traindev = X_train_traindev[m_train : ]
y_traindev = y_train_traindev[m_train : ]

# Split Dev and Test sets
m_dev = int(m_dev_test * dev_split)

X_dev = X_dev_test[: m_dev] 
y_dev = y_dev_test[: m_dev]
X_test = X_dev_test[m_dev : ]
y_test = y_dev_test[m_dev : ]

print('# Train set examples = ', X_train.shape[0])
print('# Train-Dev set examples = ', X_traindev.shape[0])
print('# Dev set examples = ', X_dev.shape[0])
print('# Test set examples = ', X_test.shape[0])

### X_raw
For v2. Dropping the traindev set here as it was not useful in v1

In [ ]:
Xraw = Xraw_features.copy()
yraw = Xraw_labels.copy()

# m_train_traindev as above

# Dev/Test sets are at the end chronologically
Xraw_train = Xraw[ : m_train_traindev]
yraw_train = yraw[ : m_train_traindev]
Xraw_dev_test  = Xraw[m_train_traindev : n_rows]
yraw_dev_test  = yraw[m_train_traindev : n_rows]

# Now can shuffle 
# idx1 = np.random.permutation(m_train_traindev)
# X_train_traindev, y_train_traindev = X_train_traindev[idx1], y_train_traindev[idx1]

idx2 = np.random.permutation(m_dev_test)
Xraw_dev_test, yraw_dev_test = Xraw_dev_test[idx2], yraw_dev_test[idx2]

# Split Train and Train/Dev sets
# m_train = int(m_train_traindev * train_split)
# X_train = X_train_traindev[ : m_train]
# y_train = y_train_traindev[ : m_train]
# X_traindev = X_train_traindev[m_train : ]
# y_traindev = y_train_traindev[m_train : ]

# Split Dev and Test sets
m_dev = int(m_dev_test * dev_split)

Xraw_dev = Xraw_dev_test[: m_dev] 
yraw_dev = yraw_dev_test[: m_dev]
Xraw_test = Xraw_dev_test[m_dev : ]
yraw_test = yraw_dev_test[m_dev : ]

print('# Train set examples = ', Xraw_train.shape[0])
# print('# Train-Dev set examples = ', Xraw_traindev.shape[0])
print('# Dev set examples = ', Xraw_dev.shape[0])
print('# Test set examples = ', Xraw_test.shape[0])

In [ ]:
# debug
Xraw_train.shape
type(Xraw_train)
Xraw_train.mean()
yraw_train.mean()

## Model -- shared

### Metrics

In [ ]:
# How often do my top-i picks end as winner ?
top1 = tf.keras.metrics.TopKCategoricalAccuracy(k=1, name="top_1", dtype=None) # = accuracy
top2 = tf.keras.metrics.TopKCategoricalAccuracy(k=2, name="top_2", dtype=None)
top3 = tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top_3", dtype=None)

### Optimizer

In [ ]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    name="Adam"
)

### TensorBoard

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard

%load_ext tensorboard
%tensorboard --logdir logs 
# top_dir = os.path.join("logs")
# log_dir = top_dir + "/fit/" #+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


## DeepHorse_v1 -- Feature-engineered model
### Create model

In [ ]:
normLayer = preprocessing.Normalization(name='normalization')
normLayer.adapt(X_train)

def make_DH(hidden_units: list, p_dropout: float=0.5, name: str=''):
    """ 
    Create 
    Input->Flatten->Dense->BN->...->Dense->Softmax
    
    Inputs -- hidden_units [u_1, ..., u_n] where u_i is the number of hidden units in FC layer i
    
    Outputs -- the model
    """
    X_input = Input(shape=X_train[0].shape)
    X = normLayer(X_input)
    X = Flatten(name='flatten')(X)
    for u in hidden_units:
        X = Dense(u, activation='relu', kernel_regularizer='l2', bias_regularizer='l2')(X)
        X = BatchNormalization()(X)
        X = Dropout(p_dropout)(X)

    X = Dense(14, activation='softmax', name='fc-softmax', kernel_regularizer='l2', bias_regularizer='l2')(X)
    mdl = Model(inputs = X_input, outputs = X, name=name)
    return mdl

In [ ]:
units = [60, 60]
DH_model = make_DH(units, 0.5) 
DH_model.summary()

In [ ]:
logdir = os.path.join("logs")
callbacks = [
#   EarlyStopping(monitor='val_loss', patience=3),
  ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', 
    save_best_only=True),
  TensorBoard(logdir, histogram_freq=1)
]

In [ ]:
X_in = X_train
y_in = y_train

DH_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy", top3])
DH_model.fit(x=X_in,
             y=y_in, 
             epochs=100,
             callbacks=callbacks,
             batch_size=64,
             verbose=2,
             shuffle=True,
             validation_data=(X_dev, y_dev)
            )

In [ ]:
DH_model.evaluate(x=X_train, y=y_train)
DH_model.evaluate(x=X_dev, y=y_dev)
DH_model.evaluate(x=X_test, y=y_test)

### v1 Hyperparameter search

In [ ]:
top_dir = os.path.join("C:\\Users\\Olaf\\OneDrive\\Work\\StanfordAI\\Project\\Data\\tf_models\\")

L = [1, 2, 3]
n = [20, 40, 60]
p = [40, 50, 60] #percent

epochs = 250

In [ ]:
i = 0
for l in L:
    for u in n: 
        for p_dropout in p:
            l_params = np.full(shape=l, fill_value=u).tolist()
            name = 'DeepHorse_v1_L' + str(l) + '_u' + str(u) + '_p' + str(p_dropout)
            dh = make_DH(l_params, p_dropout/100., name)
            dh.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy", top1, top2, top3])
            dh.fit(x=X_train, y=y_train, epochs=epochs, batch_size=32, verbose=0)
            dh.save(top_dir + name)
            i=i+1

In [ ]:
i = 0

hp_search = pd.DataFrame(columns = ['Name', 'hparams', 'Loss (Train)','Loss (Dev)', 'Accuracy (Train)', 'Accuracy (Dev)'])
for l in L:
    for u in n: 
        for p_dropout in p:        
            name = 'DeepHorse_v1_L' + str(l) + '_u' + str(u) + '_p' + str(p_dropout)
            
            df = keras.models.load_model(top_dir + name)
            res1 = np.array(df.evaluate(x=X_train, y=y_train, verbose=0))
            res2 = np.array(df.evaluate(x=X_dev, y=y_dev, verbose=0))

            h_params = str(np.full(shape=l, fill_value=u).tolist())
            res = {'Name':name, 'hparams':h_params, \
                   'Loss (Train)':res1[0], 'Loss (Dev)':res2[0], 'Accuracy (Train)':res1[1], 'Accuracy (Dev)':res2[1]}
            hp_search = hp_search.append(res, ignore_index=True)
            i = i+1 

hp_search

### General debugging space

In [ ]:
DH_model.evaluate(x=X_test[0:1], y=y_test[0:1])

In [ ]:
i = 7

X_ = X_train
y_ = y_train

for i in np.random.randint(low=0, high=1000, size=5):
    yhat = DH_model.predict(x=X_[i:i+1])
    y=y_[i:i+1]
    np.dot(yhat, y.T)
    print('y=',y, 'argmax=', np.argmax(y))
    print('yhat=',yhat, 'argmax=', np.argmax(yhat))
    print('acc=',DH_model.evaluate(x=X_[i:i+1], y=y_[i:i+1]))
    print('manual cost=',-np.dot(np.log(yhat), y.T))
    print('\n')

In [ ]:
# debug
layer_name = 'normalization'
intermediate_layer_model = keras.Model(inputs=DH_model.input,
                                       outputs=DH_model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model(X_train)
out1 = intermediate_output.numpy()

for i in range(17):
    print(out1[:,:,i].var())

(R_features[0,:,1]-R_features[0,:,1].mean())/np.sqrt(R_features[0,:,1].var())


## Sanity checking models

In [ ]:
# random 
X_noise = np.random.standard_normal(X_train.shape)
y_noise = np.random.standard_normal(y_train.shape)

DH_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy", top3])
DH_model.fit(x=X_noise,
             y=y_noise, 
             epochs=30,
             batch_size=32,
             verbose=2,
             shuffle=True,
             validation_data=(X_dev, y_dev)
            )

## Benchmark Models

### Uniform model (UM)
* Select winning horse at random
* At most 14 horses per race, so Lower bound is :
`LB = 1/14`
* Races have different number of participants so exact expectation has to be calculated by looping through races

In [ ]:
def UM_expected_accuracy(X : np.ndarray):
    """
    Calculate the expected accuracy of the Uniform Model that selects a horse at random.
    Note that as each race has a different number of entrants it has to be explicitly calculated.
    """
    expectation = 0.0
    num_races = X.shape[0]
    for i in range(num_races):
        odds_vec = X[i, :, 0]
        num_horses = len(odds_vec[odds_vec > 0.0])
        if not (num_horses > 0):
            print('race ', i)
            print(X[i, :, :])
        expectation = expectation + 1. / num_horses
    expectation = expectation / num_races
    
    return expectation

In [ ]:
print('Uniform Lower Bound = ', 1./14)
print('Uniform R = ', UM_expected_accuracy(R_features))
print('Uniform Train = ', UM_expected_accuracy(X_train))
print('Uniform Dev = ', UM_expected_accuracy(X_dev))
print('Uniform Test = ', UM_expected_accuracy(X_test))

### Public Odds model (PO)
In a pari-mutuel system (as in HKJC) the odds are determined by the amount wagered on each horse, i.e. it is a zero-sum process less a spread taken by the track.

Consequently the odds reflect the public's implied probability distribution, and the lowest odd horses are expected, by the public, to win.

In [ ]:
def Public_Odds_accuracy(X:np.ndarray, y:np.ndarray):
    """
    Public Odds model selects horse with most favorable odds
    
    Inputs:
    X -- Features assuming odds are the front
    y -- Labels in one-hot form
    """

    acc = 0.0
    num_races = X.shape[0]
    assert(num_races == y.shape[0])
    for i in range(num_races):
        odds_vec = X[i, :, 0]
        public_vec = np.zeros(shape=(14,))
        j = np.argmin(odds_vec)
       
        public_vec[j] = 1.0

        acc = acc + np.dot(public_vec, y[i])
    acc = acc / num_races
    return acc
        
def Public_Odds_accuracy_topk(X:np.ndarray, y:np.ndarray, k:int=1):
    """
    Public Odds model selects horse with most favorable odds
    
    Inputs:
    X -- Features assuming odds are the front
    y -- Labels in one-hot form
    k -- Top-k predictions (e.g. count if any top-3 prediction is winner)
    """

    acc = 0.0
    num_races = X.shape[0]
    assert(num_races == y.shape[0])
    for i in range(num_races):
        odds_vec = X[i, :, 0]
        public_vec = np.zeros(shape=(14,))

        args = np.argpartition(odds_vec, 2)
        for s in range(k):
            public_vec[args[s]] = 1.0

        acc = acc + np.dot(public_vec, y[i])
    acc = acc / num_races
    return acc
    

In [ ]:
print('Public R = ', Public_Odds_accuracy(R_features, R_labels))
print('Public Train = ', Public_Odds_accuracy(X_train, y_train))
print('Public Dev = ', Public_Odds_accuracy(X_dev, y_dev))
print('Public Test = ', Public_Odds_accuracy(X_test, y_test))

In [ ]:
print('Public R = ', Public_Odds_accuracy_topk(R_features, R_labels, 3))
print('Public Train = ', Public_Odds_accuracy_topk(X_train, y_train, 3))
print('Public Dev = ', Public_Odds_accuracy_topk(X_dev, y_dev, 3))
print('Public Test = ', Public_Odds_accuracy_topk(X_test, y_test, 3))

### Bolton-Chapman model
Historical model using a multi-nomial logit.

https://www.researchgate.net/publication/292145708_Searching_for_Positive_Returns_at_the_Track

## Analysis

### Public Odds vs Deep Horse v1

In [ ]:
def PO_vs_DH_forecasts(X:np.ndarray, y:np.ndarray):
    """
    Count times when PO and DH forecast the same winner
    
    Inputs:
    X -- Features assuming odds are the front
    y -- Labels in one-hot form
    """
    public_acc = 0.0
    dh_acc = 0.0
    public_dh_match = 0.0
    
    num_races = X.shape[0]
    assert(num_races == y.shape[0])
    for i in range(num_races):
        odds_vec = X[i, :, 0]
        public_predict = np.zeros(shape=(14,))
        j = np.argmin(odds_vec)
        public_predict[j] = 1.0
        public_acc = public_acc + np.dot(public_predict, y[i])
        
        dh_predict = np.zeros(shape=(14,))
        yhat = DH_model.predict(x=X[i:i+1])
        j = np.argmax(yhat)
        dh_predict[j] = 1.0
        dh_acc = dh_acc + np.dot(dh_predict, y[i])

        public_dh_match = public_dh_match + np.dot(dh_predict, public_predict)

    public_acc = public_acc / num_races
    dh_acc = dh_acc / num_races
    public_dh_match = public_dh_match / num_races
    return public_acc, dh_acc, public_dh_match

In [ ]:
public_acc, dh_acc, public_dh_match = PO_vs_DH_forecasts(X_dev, y_dev)

print('Public Odds accuracy = ', public_acc)
print('DH accuracy = ', dh_acc)
print('Public-DH agreement = ', public_dh_match)

In [37]:
public_acc, dh_acc, public_dh_match = PO_vs_DH_forecasts(X_test, y_test)

print('Public Odds accuracy = ', public_acc)
print('DH accuracy = ', dh_acc)
print('Public-DH agreement = ', public_dh_match)

Public Odds accuracy =  0.07775
DH accuracy =  0.1375
Public-DH agreement =  0.067


In [ ]:
public_acc, dh_acc, public_dh_match = PO_vs_DH_forecasts(X_train, y_train)

print('Public Odds accuracy = ', public_acc)
print('DH accuracy = ', dh_acc)
print('Public-DH agreement = ', public_dh_match)

## DeepHorse_v2 -- end-to-end model
### Fully connected proof-of-concept model

In [ ]:
normLayer_v2 = preprocessing.Normalization(name='normalization_v2')
normLayer_v2.adapt(Xraw_train)

def make_DH_v2(hidden_units: list, p_dropout: float=0.5, name: str=''):
    """ 
    Create 
    
    Outputs -- the model
    """
    X_input = Input(shape=Xraw_train[0].shape)
    
    # normalization is done at raw data stage
    
    X = Flatten(name='flatten')(X_input)
    for u in hidden_units:
        X = Dense(u, activation='relu', kernel_regularizer='l2', bias_regularizer='l2')(X)
        X = BatchNormalization()(X)
        X = Dropout(p_dropout)(X)

    X = Dense(14, activation='softmax', name='fc-softmax', kernel_regularizer='l2', bias_regularizer='l2')(X)
    mdl = Model(inputs = X_input, outputs = X, name=name)
    
    return mdl   
        

units = [64, 64, 32]
DH_model_v2 = make_DH_v2(units, 0.5) 
DH_model_v2.summary()


X_in = Xraw_train
y_in = yraw_train

epochs = 300

DH_model_v2.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy"])
DH_model_v2.fit(x=X_in,
                y=y_in, 
                epochs=epochs,
                batch_size=32,
                verbose=2,
                shuffle=False
                ,validation_data=(Xraw_dev, yraw_dev)
               )

### CONV2D model 
Convolutional  model for attempting on End-to-End

In [ ]:
normLayer_v2 = preprocessing.Normalization(name='normalization_v2')
normLayer_v2.adapt(Xraw_train)

def make_DH_CONV2D(filters: list, p_dropout: float=0.5, name: str=''):
    """ 
    Create 
    
    Outputs -- the model
    """
    
    new_shape = (Xraw_train.shape[1], Xraw_train.shape[2],1)
           
    F1, F2 = filters
    kernel_size = (2,10)
    
    s = 3
    
    X_input = Input(shape=new_shape)
    X = X_input
    
    
    X = Conv2D(F1, kernel_size, strides=(s,s), padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)    

    X = Conv2D(F2, kernel_size, strides=(s,s), padding='valid', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)    
        
    X = Flatten()(X)

    X = Dense(14, activation='softmax', name='fc-softmax', kernel_regularizer='l2', bias_regularizer='l2')(X)
    mdl = Model(inputs = X_input, outputs = X, name=name)
    return mdl

filters = [10, 5]
DH_model_v2 = make_DH_v2(filters, 0.5) 
DH_model_v2.summary()

X_in = Xraw_train
y_in = yraw_train

# X_noise = np.random.standard_normal(Xraw_train.shape)
# y_noise = np.random.standard_normal(yraw_train.shape)

epochs = 5

DH_model_v2.compile(loss='categorical_crossentropy', optimizer=opt, metrics = ["accuracy"])
DH_model_v2.fit(x=X_in,
                y=y_in, 
                epochs=epochs,
                batch_size=32,
                verbose=2,
                shuffle=False,
                validation_data=(Xraw_dev, yraw_dev)
               )